# Functions and imports

In [1]:
import time
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from pandas.plotting import scatter_matrix
import os
import re
import pandas as pd
import numpy as np
from IPython.display import Audio
import seaborn as sns

import helpers # this is where the main training/decoding functions are, modified from teh original HIVAE main.py

#import warnings 
#warnings.filterwarnings('ignore') ########## NOTE: comment out for testing in case it's hiding problems

def set_settings(opts,nepochs=500,modload=False,save=True): # note: modload doesnt do anything right now, hardcoded in helpers.py
    'replace setting template placeholders with file info'
    inputf=re.sub('.csv','',opts['files'].iloc[0])
    #missf=inputf+'_missing.csv'
    typef=inputf+'_types.csv'
    
    template = '--epochs NEPOCHS --model_name model_HIVAE_inputDropout --restore MODLOAD \
        --data_file data_python_Notmissing/INPUT_FILE.csv --types_file data_python_Notmissing/TYPES_FILE \
         --batch_size NBATCH --save NEPFILL --save_file SAVE_FILE\
        --dim_latent_s SDIM --dim_latent_z 1 --dim_latent_y YDIM \
        --learning_rate LRATE'
    
    # replace placeholders in template
    settings = re.sub('INPUT_FILE',inputf,template)
    settings = re.sub('NBATCH',str(opts['nbatch'].iloc[0]),settings)
    settings = re.sub('NEPOCHS',str(nepochs),settings)
    settings = re.sub('NEPFILL',str(nepochs-1),settings) if save else re.sub('NEPFILL',str(nepochs*2),settings)
    settings = re.sub('YDIM',str(opts['ydims'].iloc[0]),settings)
    settings = re.sub('SDIM',str(opts['sdims'].iloc[0]),settings)
    #settings = re.sub('MISS_FILE',missf,settings) #if not 'medhist' in inputf else re.sub('--true_miss_file data_python/MISS_FILE','',settings)
    settings = re.sub('TYPES_FILE',typef,settings)
    settings = re.sub('SAVE_FILE',inputf,settings)
    settings = re.sub('LRATE',str(opts['lrates'].iloc[0]),settings)
    settings = re.sub('MODLOAD','1',settings) if modload else re.sub('MODLOAD','0',settings)
    
    return settings

In [2]:
sample_size=178
# get file list
files=[i for i in os.listdir('data_python_NotMissing/') if not '_type' in i and not '_missing' in i and not '.DS_Store' in i and not 'stalone' in i]
files.sort()
print(files)

['ARGlobalTelemetryVariance.csv', 'ARIntroReadTimes.csv', 'ARObjectPlacement.csv', 'ARObjectPlacementFFT.csv']


In [3]:
best_hyper = pd.read_csv('best_hyper_ALTOIDA_processed_nm.csv')
best_hyper

,lrates,nbatch,wdecay,ydims,files,loss,sdims
0,0.010,16,0.001,1,ARGlobalTelemetryVariance.csv,-15.173678,1
1,0.010,32,0.000,1,ARIntroReadTimes.csv,16.816938,2
2,0.010,16,0.001,1,ARObjectPlacement.csv,30.772295,1
3,0.001,16,0.001,1,ARObjectPlacementFFT.csv,107.652156,1


# VP decoding

Run after bnet.R

In [4]:
os.getcwd()

'/Users/meemansasood/Documents/Documents_IT/paper/Altoida_VAMBN_paper/data/HI-VAE'

In [5]:
meta = pd.read_csv('metaenc_nm.csv')
meta

,scode_ARGlobalTelemetryVariance,SUBJID,zcode_ARGlobalTelemetryVariance,scode_ARIntroReadTimes,zcode_ARIntroReadTimes,scode_ARObjectPlacement,zcode_ARObjectPlacement,scode_ARObjectPlacementFFT,zcode_ARObjectPlacementFFT
0,0,009-20_001_1,-0.445833,0,-0.852945,0,1.240026,0,1.631415
1,0,009-20_002_1,0.377560,1,2.410406,0,0.710988,0,0.002221
2,0,009-20_003_1,0.880156,1,0.403134,0,-0.642160,0,0.248468
3,0,009-20_004_1,-0.464829,1,1.869062,0,0.325809,0,0.325940
4,0,009-20_005_1,-0.941436,1,0.786361,0,0.680686,0,1.291635
...,...,...,...,...,...,...,...,...,...
173,0,005_012_2,1.128772,1,0.581977,0,0.101907,0,-0.659471
174,0,005_012_3,1.055777,0,-3.205861,0,-1.286230,0,-0.081339
175,0,005_015_1,1.812891,0,-1.927395,0,-0.988188,0,2.691695
176,0,005_015_2,0.645875,0,-2.775221,0,0.576342,0,5.778634


In [6]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/paper/Altoida_VAMBN_paper/data/data_out/main_VirtualPPts_nm.csv')
VPcodes

,zcode_ARGlobalTelemetryVariance,scode_ARIntroReadTimes,zcode_ARIntroReadTimes,zcode_ARObjectPlacement,zcode_ARObjectPlacementFFT
0,-1.482899,1,4.246285,2.068885,-1.640155
1,-0.192269,1,0.749230,-0.479894,-2.030838
2,-0.186759,1,0.976250,0.586558,1.421124
3,0.653953,1,2.114233,0.242126,-0.458918
4,0.141366,1,0.979570,0.491061,-1.134798
...,...,...,...,...,...
173,-0.062742,0,-3.042447,-0.947173,0.914997
174,-0.157621,0,-2.586972,0.394617,1.181893
175,-0.630218,0,-2.662090,-0.289942,-2.789332
176,0.084332,0,-1.793706,-0.021602,-0.266958


In [ ]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/paper/Altoida_VAMBN_paper/data/data_out/main_VirtualPPts_nm.csv')

dfs=list()
virt=list()
for f in files:
    # replace placeholders in template
    opts=dict(best_hyper[best_hyper['files'].copy()==f])
    opts['nbatch'].iloc[0]=sample_size
    settings=set_settings(opts,nepochs=1,modload=True,save=False)
    
    #run
    zcodes=VPcodes['zcode_'+re.sub('.csv','',f)]
    scodes=VPcodes['scode_'+re.sub('.csv','',f)] if 'scode_'+re.sub('.csv','',f) in VPcodes.columns else np.zeros(zcodes.shape)
        
    dec=helpers.dec_network(settings,zcodes,scodes,VP='nomiss')
    subj=pd.read_csv('python_names_Notmissing/'+re.sub('.csv','',f)+'_subj.csv')['x']
    names=pd.read_csv('python_names_Notmissing/'+re.sub('.csv','',f)+'_cols.csv')['x']
    dat=pd.DataFrame(dec)
    dat.columns=names
    dat['SUBJID']=subj
    virt.append(dec)
    dfs.append(dat)

virt_dic=dict(zip(files,virt))
decoded=helpers.merge_dat(dfs)
decoded.to_csv('decodedVP_nm.csv',index=False)

Get Loglikelihoods for R plot!

In [7]:
VPcodes = pd.read_csv('/Users/meemansasood/Documents/Documents_IT/paper/Altoida_VAMBN_paper/data/data_out/main_VirtualPPts.csv')

dfs=list()
for f in files:
    # replace placeholders in template
    opts=dict(best_hyper[best_hyper['files'].copy()==f])
    opts['nbatch'].iloc[0]=sample_size
    settings=set_settings(opts,nepochs=1,modload=True,save=False)
    
    #run
    zcodes=VPcodes['zcode_'+re.sub('.csv','',f)]
    scodes=VPcodes['scode_'+re.sub('.csv','',f)] if 'scode_'+re.sub('.csv','',f) in VPcodes.columns else np.zeros(zcodes.shape)
        
    loglik=helpers.dec_network_loglik(settings,zcodes,scodes,VP='nomiss')
    loglik=np.nanmean(np.array(loglik).T,axis=1)
    subj=pd.read_csv('python_names_Notmissing/'+re.sub('.csv','',f)+'_subj.csv')['x']
    dat=pd.DataFrame(loglik)
    dat.columns=[f]
    dat['SUBJID']=subj
    dfs.append(dat)

decoded=helpers.merge_dat(dfs)
decoded.to_csv('virtual_logliks_nm.csv',index=False)

Namespace(batch_size=178, data_file='data_python_Notmissing/ARGlobalTelemetryVariance.csv', dim_latent_s=1, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.01, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='ARGlobalTelemetryVariance', true_miss_file=None, types_file='data_python_Notmissing/ARGlobalTelemetryVariance_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders




/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


[*] Defining Encoder...
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.


[*] Defining Decoder...

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf

/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Namespace(batch_size=178, data_file='data_python_Notmissing/ARIntroReadTimes.csv', dim_latent_s=2, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.01, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='ARIntroReadTimes', true_miss_file=None, types_file='data_python_Notmissing/ARIntroReadTimes_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARIntroReadTimes/ARIntroReadTimes.ckpt
Model restored: ./Saved_Networks/ARIntroReadTimes/ARIntroReadTimes.ckpt
::::::DECODING:::::::::
:::::::::::: ones for miss list VP


/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Namespace(batch_size=178, data_file='data_python_Notmissing/ARObjectPlacement.csv', dim_latent_s=1, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.01, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='ARObjectPlacement', true_miss_file=None, types_file='data_python_Notmissing/ARObjectPlacement_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARObjectPlacement/ARObjectPlacement.ckpt
Model restored: ./Saved_Networks/ARObjectPlacement/ARObjectPlacement.ckpt
::::::DECODING:::::::::
:::::::::::: ones for miss list VP


/Users/meemansasood/Library/Python/3.7/lib/python/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


Namespace(batch_size=178, data_file='data_python_Notmissing/ARObjectPlacementFFT.csv', dim_latent_s=1, dim_latent_y=1, dim_latent_y_partition=None, dim_latent_z=1, display=1, epochs=1, learning_rate=0.001, miss_file='Missing_test.csv', miss_percentage_test=0.0, miss_percentage_train=0.0, model_name='model_HIVAE_inputDropout', perp=10, plot=1, restore=1, save=2, save_file='ARObjectPlacementFFT', true_miss_file=None, types_file='data_python_Notmissing/ARObjectPlacementFFT_types.csv')
[*] Importing model: model_HIVAE_inputDropout
[*] Defining placeholders
[*] Defining Encoder...
[*] Defining Decoder...
[*] Defining Cost function...
INFO:tensorflow:Restoring parameters from ./Saved_Networks/ARObjectPlacementFFT/ARObjectPlacementFFT.ckpt
Model restored: ./Saved_Networks/ARObjectPlacementFFT/ARObjectPlacementFFT.ckpt
::::::DECODING:::::::::
:::::::::::: ones for miss list VP
